In [22]:
import numpy as np
from collections import namedtuple
from ortools.sat.python import cp_model
import timeit

In [3]:
''' Algorithmic Approach : use solve_schedule_algo(intervals, num_resources) '''

# Proof: http://www.cs.toronto.edu/~milad/csc373/lectures/T1.pdf
def solve_schedule_algo(intervals, num_resources):
  sorted_end   = sorted(intervals, key=lambda x: x[1])    # n*log(n)
  schedules = [[] for _ in range(num_resources)]

  for interval in sorted_end: 
    # get schedule with smallest end time such that the start time of interval is greater than endtime
    index = assign_schedule(interval[0], schedules)
    if index > -1: schedules[index].append(interval)

  return sum([len(schedule) for schedule in schedules]), schedules  

def assign_schedule(start_time, schedules):
  index = -1
  latest_end = -1
  for i, schedule in enumerate(schedules): 
    schedule_end_time = 0 if len(schedule) < 1 else schedule[-1][1]
    if schedule_end_time > start_time:
      continue
    elif schedule_end_time > latest_end:
      latest_end = schedule_end_time
      index = i
  return index

In [4]:
''' SMT Approach : use solve_schedule_smt(intervals, num_resources) '''

def solve_schedule_smt(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Maximize Length of Schedule
  model.Maximize(sum(assign_resource.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return solver.ObjectiveValue(), solution

In [5]:
''' SMT Approach + Resource Optimization : use solve_schedule_opt(intervals, num_resources) '''

def solve_schedule_opt(intervals, num_resources):
  # Solve Intermediate Solution
  obj = int(solve_schedule(intervals, num_resources)[0])

  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Add Optimal Value for Solver 2
  model.Add(sum(assign_resource.flatten()) == obj)
  model.Maximize(cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return obj, solution, solver.ObjectiveValue()

In [6]:
''' Multiple-Objective Optimization Static Weighted : solve_static_weighted(intervals, num_resources) '''

def solve_static_weighted(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(10000*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [7]:
''' Multiple-Objective Optimization Dynamic Weighted : solve_dynamic_weighted(intervals, num_resources) '''

def solve_dynamic_weighted(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # Grab Maximum Range of Intervals
  flattened = list(sum(intervals,()))
  task_weights = max(flattened) - min(flattened)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(task_weights*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [8]:
def cumulative_time(schedules): 
  total_time = 0
  for intervals in schedules: 
    for i in intervals: 
      total_time += i[1]-i[0]
  return total_time

def display_results(schedules): 
  for i, s in enumerate(schedules):
    print(f'Resource {i+1}:', s)

In [11]:
def check_testcase(testcase, approach1, approach2, output=True):
  output1 = approach1(*testcase)
  output2 = approach2(*testcase)
  sol1, sch1 = output1[0], output1[1]
  sol2, sch2 = output2[0], output2[1]

  # Ensure that optimal value matches 
  assert(sol1 == sol2)

  if output:
    print(f'Intervals: {str(testcase.intervals)}\nNumber of Resources: {testcase.num_resources}\n')
    print(f'Approach 1\tTotal Time: {cumulative_time(sch1)}')
    display_results(sch1)
    print('--------')
    print(f'Approach 2\tTotal Time: {cumulative_time(sch2)}')
    display_results(sch2)

testcase = namedtuple('testcase', ['intervals', 'num_resources'])

In [20]:
def get_runtime(approach, intervals, num_resources):
  start = timeit.default_timer()
  approach(intervals, num_resources)
  end = timeit.default_timer()
  return end-start

In [48]:
intervals = [(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)]
num_resources = 2

# check_testcase(intervals, num_resources)
print(solve_schedule_opt(intervals, num_resources))
print(solve_static_weighted(intervals, num_resources))

intervals = [(1,3), (3,5), (5,7), (1,1000000000000), (2, 4), (4, 6), (6, 11), (2, 1000000000000)]
num_resources = 2

# check_testcase(intervals, num_resources)
print(solve_schedule_opt(intervals, num_resources))
print(solve_static_weighted(intervals, num_resources))
print(solve_dynamic_weighted(intervals, num_resources))

(5, 17.0, [[(1, 3), (4, 11)], [(2, 5), (6, 8), (9, 12)]])
(5, 50017.0, [[(1, 3), (4, 11)], [(2, 5), (6, 8), (9, 12)]])
(6, 15.0, [[(1, 3), (3, 5), (5, 7)], [(2, 4), (4, 6), (6, 11)]])
(2, 2000000019997.0, [[(1, 1000000000000)], [(2, 1000000000000)]])
(6, 6000000000009.0, [[(1, 3), (3, 5), (5, 7)], [(2, 4), (4, 6), (6, 11)]])


In [24]:
testcases = [
  testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 1),
  testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 2), 
  testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 3),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 1),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 2),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 3),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 4),
  testcase([(1,10), (1,15), (11,20), (16,20), (15,21)], 2),
  testcase([(91, 683), (393, 952), (130, 242), (182, 450), (289, 564), (197, 198), (0, 694), (427, 435), (638, 1387), (619, 652), (24, 119), (1193, 1436), (455, 538), (203, 885), (683, 1339), (745, 1265), (395, 983), (541, 711), (963, 1238), (192, 831), (682, 879), (55, 409), (53, 117), (220, 251), (495, 887), (135, 265), (85, 324), (126, 400), (893, 1129), (1029, 1430), (1073, 1234), (102, 616), (193, 199), (258, 885), (72, 138), (6, 57), (88, 106), (94, 632), (562, 614), (328, 695), (213, 328), (538, 855), (297, 914), (124, 141), (84, 214), (986, 1278), (549, 1343), (346, 1309), (76, 93), (364, 732), (392, 410), (1027, 1308), (493, 586), (194, 1151), (58, 65), (220, 1289), (900, 1188), (399, 1213), (33, 39), (198, 382), (27, 238), (157, 437), (577, 598), (285, 531), (2, 20), (305, 1282), (1039, 1422), (1010, 1052), (0, 14), (531, 815), (263, 459), (80, 212), (15, 1274), (170, 615), (42, 283), (312, 509), (129, 164), (137, 1358), (495, 1186), (393, 401), (31, 555), (734, 740), (1109, 1263), (541, 1366), (31, 51), (420, 1143), (372, 1017), (254, 1097), (234, 1326), (67, 73), (53, 586), (42, 424), (445, 548), (394, 1069), (926, 1004), (7, 12), (264, 1182), (170, 1363), (189, 778), (262, 1022), (11, 55), (695, 1080), (575, 926), (748, 919), (142, 1376), (188, 733), (352, 508), (170, 274), (404, 461), (270, 351), (382, 1113)], 4)
]
for case in testcases: 
  check_testcase(case, solve_schedule_algo, solve_schedule_smt, False)
  print('Algorithmic Runtime: ', get_runtime(solve_schedule_algo, *case))
  print('SMT Synthesis Runtime: ', get_runtime(solve_schedule_smt, *case))
  print('\n.....\n')


Algorithmic Runtime:  2.4484999812557362e-05
SMT Synthesis Runtime:  0.014256632000069658

.....

Algorithmic Runtime:  2.948200017272029e-05
SMT Synthesis Runtime:  0.011717137999767147

.....

Algorithmic Runtime:  3.585200011002598e-05
SMT Synthesis Runtime:  0.021044983000138018

.....

Algorithmic Runtime:  2.8795999696740182e-05
SMT Synthesis Runtime:  0.013609896000161825

.....

Algorithmic Runtime:  3.164899999319459e-05
SMT Synthesis Runtime:  0.007858554999984335

.....

Algorithmic Runtime:  3.874000003634137e-05
SMT Synthesis Runtime:  0.02238664999958928

.....

Algorithmic Runtime:  2.797599972836906e-05
SMT Synthesis Runtime:  0.011614451999776065

.....

Algorithmic Runtime:  2.511000002414221e-05
SMT Synthesis Runtime:  0.006441584999720362

.....



In [12]:
# testcase([(9,10:50), (9:30,11:45), (1, 2:50), (3-4:50), (11-12:50), (6, 6:50)])

check_testcase(testcase([(540, 650), (570, 705), (780, 890), (900, 1010), (660, 770), (1080, 1140)], 1), solve_schedule_algo, solve_schedule_smt)

Intervals: [(540, 650), (570, 705), (780, 890), (900, 1010), (660, 770), (1080, 1140)]
Number of Resources: 1

Approach 1	Total Time: 500
Resource 1: [(540, 650), (660, 770), (780, 890), (900, 1010), (1080, 1140)]
--------
Approach 2	Total Time: 500
Resource 1: [(540, 650), (780, 890), (900, 1010), (660, 770), (1080, 1140)]


In [23]:
intervals = [(540, 650), (570, 705), (780, 890), (900, 1010), (660, 770), (1080, 1140)]
num_resources = 1

print('Algorithmic Runtime: ', get_runtime(solve_schedule_algo, intervals, num_resources))
print('SMT Synthesis Runtime: ', get_runtime(solve_schedule_smt, intervals, num_resources))


Algorithmic Runtime:  3.980900009992183e-05
SMT Synthesis Runtime:  0.004933660000006057
